In [1]:
class CallCount:
    def __init__(self,f):
        self.f = f
        self.count = 0
    def __call__(self,*args,**kwargs):
        self.count += 1
        return self.f(*args,**kwargs)
# class object itself act as decorator
@CallCount
def hello(name):
    return 'hello {} !'.format(name)


In [2]:
hello('Eric')

'hello Eric !'

In [3]:
hello('Mary')

'hello Mary !'

In [4]:
hello('Buddy')

'hello Buddy !'

In [5]:
CallCount.count

AttributeError: type object 'CallCount' has no attribute 'count'

In [6]:
hello.count

3

In [8]:
class Trace:
    def __init__(self):
        self.enabled = True
    def __call__(self,f):
        def wrap(*args,**kwargs):
            if self.enabled:
                print("Calling {}".format(f))
            return f(*args,**kwargs)
        return wrap
tracer = Trace()
@tracer
def rotate_list(l):
    return l[1:] + [l[0]]

num = [1,2,3,4]
rotate_list(num)

Calling <function rotate_list at 0x0000000005002708>


[2, 3, 4, 1]

In [13]:
num=rotate_list(num)
num

Calling <function rotate_list at 0x0000000005002708>


[1, 2, 3, 4]

In [16]:
def escape_unicode(f):
    def wrap(*args,**kwargs):
        x=f(*args,**kwargs)
        return ascii(x)
    return wrap

In [17]:
tracer = Trace()
@tracer         # runs last
@escape_unicode # runs first
def island(name):
    return name + '0y'

class IslandMaker:
    def __init__(self,suffix):
        self.suffix = suffix
        
        
    @tracer
    def make_island(self,name):
        return name + self.suffix
im = IslandMaker(' Island')
im.make_island('Lama')

Calling <function IslandMaker.make_island at 0x0000000007F9D3A8>


'Lama Island'

In [24]:
import functools
def noop(f):
    @functools.wraps(f)
    def noop_wrapper():
        return f()
#    noop_wrapper.__name__ = f.__name__
#    noop.wrapper.__doc__ = f.__doc__
    return noop_wrapper

@noop
def hello():
    """Doc string - Hello World"""
    print('Hello, World')
fn=hello()
help(hello)

Hello, World
Help on function hello in module __main__:

hello()
    Doc string - Hello World



In [26]:
def check_non_negative(index):
    def validator(f):
        def wrap(*args):
            if args[index] < 0:
                raise ValueError("Argument {} must be Non-Negative".format(index))
            return f(*args)
        return wrap
    return validator

@check_non_negative(1)
def create_list(value,size):
    return [value] *size
create_list(3,4)

[3, 3, 3, 3]

In [27]:
create_list(3,-4)

ValueError: Argument 1 must be Non-Negative